In [187]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
import random

# 1. Подготавливаем webdriver

In [14]:
#Service(ChromeDriverManager().install())

In [217]:
driver = webdriver.Chrome()
# Ждем, пока браузер загрузится
time.sleep(20)

# 2. Собираем список ноутбуков

Объявляем вспомогательную функцию для скрапинга раздела со списком ноутбуков

In [203]:
def scrap_laptop_list(driver, depth=10, verbose=False):
    laptop_list = pd.DataFrame(columns=['id', 'model', 'specs', 'rating', 'votes', 'service_rating', 'price', 'price_discount', 'url'])
    
    for i in range(1, depth + 1):
        if verbose:
            print('Обрабатываем страницу номер', i)
            
        # Запрашиваем страницу с порядковым номером i
        url = 'https://www.dns-shop.ru/catalog/17a892f816404e77/noutbuki/?p=' + str(i)
        driver.get(url)
        # Ждем, чтобы страница успела загрузиться
        if i == 1:
            time.sleep(15)
        else:
            time.sleep(10)
        
        # Вытаскиваем нужный div со списком ноутбуков и передаем парсеру
        catalog_element = driver.find_element(By.XPATH, "//div[@class='catalog-products view-simple']")
        catalog_html = catalog_element.get_attribute('innerHTML')
        bs = BeautifulSoup(catalog_html, 'html.parser')
        
        # Парсим список ноутбуков
        for element in bs:
            if element.name == 'div':
                                    
                # Вытаскиваем ID ноутбука
                id = element['data-product']
                #print(id)
                
                # Вытаскиваем URL
                url_tag = element.find('a', class_='catalog-product__name ui-link ui-link_black')
                url = 'https://www.dns-shop.ru' + url_tag['href']
                #print(url)
                
                # Вытаскиваем модель
                model_tag = url_tag.findChildren()[0]            
                model = model_tag.text.split('[')[0].strip()
                #print(model)
        
                # Вытаскиваем характеристики
                specs = model_tag.text.split('[')[1].strip(']').replace(' , ', ', ').replace(',', ';')
                #print(specs)
                
                # Вытаскиваем рейтинг
                rating_tag = element.find('a', class_='catalog-product__rating ui-link ui-link_black')
                rating = float(rating_tag['data-rating'])
                #print(rating)
                
                # Вытаскиваем число проголосовавших
                try:
                    votes = int(rating_tag.text)
                except ValueError:
                    votes = 0
                #print(votes)
                
                # Вытаскиваем рейтинг надежности
                service_rating_tag = element.find('a', class_='catalog-product__service-rating ui-link ui-link_black')
                if service_rating_tag is not None:
                    service_rating = float(service_rating_tag.text.replace(',', '.'))
                else:
                    service_rating = 0
                #print(service_rating)
                
                # Вытаскиваем цену и цену со скидкой
                price_tag = element.find('div', class_='product-buy__price')
                price_history = price_tag.text.replace(' ', '').split('₽')
                if price_history[1] == '':
                    price = int(price_history[0])
                else:
                    price = int(price_history[1])
                price_discount = int(price_history[0])
                #print(price, price_discount)
                        
                laptop_list.loc[len(laptop_list.index)] = [
                    id,
                    model,
                    specs,
                    rating,
                    votes,
                    service_rating,
                    price,
                    price_discount,
                    url
                ]
        
    return laptop_list

In [20]:
laptop_list = scrap_laptop_list(driver, 41)

In [179]:
laptop_list

,id,model,specs,rating,votes,service_rating,price,price_discount,url
0,829ba653-24bd-11ec-8ef9-00155d8ed20b,"13.3"" Ноутбук Irbis NB77 черный",HD (1366x768); TN+film; Intel Atom Z3735F; ядр...,3.5,89,93.11,14999,14999,https://www.dns-shop.ru/product/829ba65324bded...
1,e259ba0d-eda1-11eb-a293-00155dd2ff2c,"14"" Ноутбук Irbis NB257 серый",HD (1366x768); IPS; Intel Celeron 3350; ядра: ...,4.5,52,97.67,15999,15999,https://www.dns-shop.ru/product/e259ba0deda12f...
2,ce257262-3e34-11ed-9025-00155d8ed20b,"14.1"" Ноутбук Irbis NB270 белый",Full HD (1920x1080); IPS; Intel Celeron N4020;...,4.0,27,98.70,15999,15999,https://www.dns-shop.ru/product/ce2572623e34ed...
3,c453df98-dd51-4fd2-a984-7b3bcd0a531f,"14.1"" Ноутбук Digma EVE 14 C414 серый",Full HD (1920x1080); IPS; Intel Celeron N4020;...,2.5,3,97.76,17999,17999,https://www.dns-shop.ru/product/c453df98dd51a5...
4,9e7bf670-f005-4192-918d-471de2c20659,"14"" Ноутбук Irbis NB283 серый",HD (1366x768); IPS; Intel Celeron 3350; ядра: ...,0.0,0,0.00,19199,19199,https://www.dns-shop.ru/product/9e7bf670f00520...
...,...,...,...,...,...,...,...,...,...
709,23df53b5-eb7a-11ec-8fd3-00155d8ed20b,"17.3"" Ноутбук MSI GE77 HX Raider 12UHS-085RU с...",Quad HD 2K (2560x1440); IPS; Intel Core i9-129...,4.5,21,96.83,299999,254999,https://www.dns-shop.ru/product/23df53b5eb7aed...
710,891e4b9c-5e0c-11ec-8f49-00155d8ed20b,"13.4"" Ноутбук ASUS ROG Flow Z13 GZ301ZE-LD226W...",1920x1200; IPS; Intel Core i9-12900H; ядра: 6 ...,5.0,1,0.00,284999,256499,https://www.dns-shop.ru/product/891e4b9c5e0ced...
711,2a13b453-3ef1-11ed-9027-00155d8ed20c,"17.3"" Ноутбук MSI CreatorPro Z17 A12UKST-259RU...",2560x1600; IPS; Intel Core i7-12700H; ядра: 6 ...,0.0,0,0.00,269999,269999,https://www.dns-shop.ru/product/2a13b4533ef1ed...
712,ec24fcab-5ce5-4ba4-a163-f9d440908b48,"16"" Ноутбук MSI CreatorPro Z16P B12UKST-222RU ...",2560x1600; IPS; Intel Core i7-12700H; ядра: 6 ...,0.0,0,0.00,274999,274999,https://www.dns-shop.ru/product/ec24fcab5ce508...


Выгружаем список ноутбуков

In [22]:
laptop_list.to_csv('dns_laptop_list.csv', index=False)

# 3. Собираем характеристики ноутбуков

Объявляем вспомогательную функцию для скрапинга страницы ноутбука

In [222]:
def scrap_laptop_specs(driver, laptop_list):
    laptop_dataset_list = list()

    try:
        c = 1
        for index, row in laptop_list.iterrows():
            print('Обработка %d/%d: %s' % (c, laptop_list.shape[0], row.model))
    
            # Инициализируем словарь с характеристиками
            spec_dict = {
                'id': row.id,
                'rating': float(row.rating),
                'votes': int(row.votes),
                'service_rating': float(row.service_rating),
                'price': int(row.price),
                'price_discount': int(row.price_discount),
                'url': row.url,
            }
    
            # Запрашиваем страницу и ждем пока она загрузится
            driver.get(row.url)
            if c%30 == 0:
                time.sleep(60 + random.randint(1, 60))
            elif c%230 == 0:
                time.sleep(5*60 + random.randint(60, 120))
            else:
                time.sleep(10 + random.randint(1, 5))
                
    
            # Находим раздел с характеристиками
            specs_element = driver.find_element(By.XPATH, "//div[@class='product-characteristics product-characteristics_collapsed']")
            specs_html = specs_element.get_attribute('innerHTML')
            bs = BeautifulSoup(specs_html, 'html.parser')
    
            # Вытаскиваем все характеристики и складываем в словарь
            for group_tag in bs:
                for spec_tag in group_tag:        
                    if 'product-characteristics__spec' in spec_tag['class']:
                        spec_title_tag = spec_tag.find('div', class_='product-characteristics__spec-title')
                        spec_value_tag = spec_tag.find('div', class_='product-characteristics__spec-value')
                        spec_dict[spec_title_tag.text.strip()] = spec_value_tag.text.strip()
    
            # Добавляем словарь с характеристиками в результирующий список
            laptop_dataset_list.append(spec_dict)
    
            c += 1
    finally:
        return laptop_dataset_list

In [ ]:
laptop_dataset_list = scrap_laptop_specs(driver, laptop_list)

In [ ]:
laptop_dataset_list

Конвертируем список из словарей в датасет

In [170]:
def dict_list_to_dataframe(dict_list):
    result = pd.DataFrame()
    for d in dict_list:
        result = pd.concat([result, pd.Series(d).to_frame().T])
    return result

In [171]:
laptop_dataset = dict_list_to_dataframe(laptop_dataset_list)

In [ ]:
laptop_dataset

In [173]:
laptop_dataset.to_csv('dns_laptop_dataset.csv', index=False)

# 4. Закрываем webdriver

In [174]:
driver.quit()